In [1]:
import torch
from copy import deepcopy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer



C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset('ag_news')
train_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
validation_test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(1000))])
validation_ds = Dataset.from_dict(validation_test_ds[0:500])
test_ds = Dataset.from_dict(validation_test_ds[500:1000])

data = test_ds





Using custom data configuration default
Reusing dataset ag_news (C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 504.49it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-12f3c4e4bf422cce.arrow


In [3]:
display(data)
display(data[0:5])

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

{'text': ['A Sneak Peek at Trillian 3.0 Instant Messaging The popular IM consolidation service adds audio and video chat.',
  'New Technology Powers Fuel Cells A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around \\$90, its Japanese inventors claim.',
  "Court rules against state Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.S. federal court on free-speech grounds.",
  'Braves, Smoltz agree to new deal Atlanta, GA (Sports Network) - The Atlanta Braves announced Thursday that the team has come to terms with longtime pitcher John Smoltz to a new two-year contract with a club option for the 2007 season.',
  "US Supreme Court asked to rule on homosexuals' right to adopt children (AFP) AFP - The American Civil Liberties Union, the leading US civil rights group, 

In [4]:
import textattack
dataset = textattack.datasets.HuggingFaceDataset(data)
display(dataset)

In [5]:
#attack the original model using TextFooler
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model_path = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\CheckList\run3'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=4)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)



In [6]:
from textattack.attack_recipes.pwws_ren_2019 import PWWSRen2019
attack = PWWSRen2019.build(model_wrapper)
attack.cuda_()
display(attack)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)

In [7]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=-1, random_seed = 765,
                         log_to_csv = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\CheckList\pwws_attack_results\run3\results.csv', 
                         csv_coloring_style = 'file')

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


textattack: Logging to CSV at path C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\CheckList\pwws_attack_results\run3\results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



  0%|▏                                                                                 | 1/500 [00:02<20:31,  2.47s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 1/500 [00:02<22:29,  2.70s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[3 (97%)]] --> [[2 (56%)]]

[[A]] [[Sneak]] Peek at Trillian 3.[[0]] [[Instant]] Messaging The [[popular]] IM consolidation service [[adds]] audio and [[video]] [[chat]].

[[adenine]] [[fink]] Peek at Trillian 3.[[nought]] [[inst]] Messaging The [[democratic]] IM consolidation service [[lend]] audio and [[television]] [[confab]].




[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                            | 2/500 [00:03<15:50,  1.91s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                            | 2/500 [00:03<15:55,  1.92s/it]

...

--------------------------------------------- Result 497 ---------------------------------------------
[[2 (96%)]] --> [[0 (52%)]]

Vivendi, Messier Are Fined \$1.35M Each [[French]] [[regulators]] fined Vivendi [[Universal]] [[SA]] and its former boss Jean-Marie Messier nearly \$1.4 million each on Tuesday for deceiving investors with a litany of inaccurate financial communications issued over a two-year period.

Vivendi, Messier Are Fined \$1.35M Each [[Gallic]] [[governor]] fined Vivendi [[general]] [[Sturmabteilung]] and its former boss Jean-Marie Messier nearly \$1.4 million each on Tuesday for deceiving investors with a litany of inaccurate financial communications issued over a two-year period.




C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 298 / 158 / 42 / 498: 100%|██████████████████▉| 498/500 [13:00<00:03,  1.57s/it]

--------------------------------------------- Result 498 ---------------------------------------------
[[1 (88%)]] --> [[0 (52%)]]

Vieira [[slams]] Spain #39;s stance on [[racism]] Patrick Vieira has fiercely criticised Spain #39;s [[record]] in combating [[racism]] in football and paid tribute to England #39;s [[record]] in acting against it.

Vieira [[mosh]] Spain #39;s stance on [[racialism]] Patrick Vieira has fiercely criticised Spain #39;s [[memorialise]] in combating [[racialism]] in football and paid tribute to England #39;s [[commemorate]] in acting against it.




[Succeeded / Failed / Skipped / Total] 298 / 158 / 42 / 498: 100%|██████████████████▉| 499/500 [13:04<00:01,  1.57s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 298 / 159 / 42 / 499: 100%|██████████████████▉| 499/500 [13:04<00:01,  1.57s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 298 / 159 / 43 / 500: 100%|███████████████████| 500/500 [13:04<00:00,  1.57s/it]

--------------------------------------------- Result 499 ---------------------------------------------
[[0 (98%)]] --> [[[FAILED]]]

Parliament passes no-confidence bid NAIROBI, Kenya -- Somalia's parliament passed a motion of no-confidence against the new prime minister and his Cabinet yesterday, an official said, effectively sacking a government that had been expected to restore order to the country after 13 years of anarchy and war. The deputy speaker of the 275-member transitional parliament, Dalhar Omar, said 153 members voted against Prime Minister ...


--------------------------------------------- Result 500 ---------------------------------------------
[[2 (81%)]] --> [[[SKIPPED]]]

Yahoo! Tunes In to Musicmatch Close watchers of the online music business no doubt noted yesterday #39;s announcement of Yahoo! #39;s (Nasdaq: YHOO) purchase of Musicmatch with interest.



+-------------------------------+--------+
| Attack Results                |        |
+----------------------

In [8]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))
results_html = logger.df[['original_text', 'perturbed_text']].to_html(escape=False)
path = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\CheckList\pwws_attack_results\run3\results.html'
text_file = open(path, "w")
text_file.write(results_html.encode("gbk", 'ignore').decode("gbk", "ignore"))
text_file.close()
logger.flush()

textattack: Logging to CSV at path results.csv
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated 

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,original_text,perturbed_text
0,A Sneak Peek at Trillian 3.0 Instant Messaging The popular IM consolidation service adds audio and video chat.,adenine fink Peek at Trillian 3.nought inst Messaging The democratic IM consolidation service lend audio and television confab.
1,"New Technology Powers Fuel Cells A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around \$90, its Japanese inventors claim.","New Technology index Fuel Cells angstrom new fuel cell for notebook PCs, more contract and powerful than competing technologies, could be on the market in betimes 2006 at a price of round \$90, its Japanese inventors claim."
2,Court rules against state Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.S. federal court on free-speech grounds.,judicature dominion against say Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.siemens. Union judicature on free-speech grounds.
3,"Braves, Smoltz agree to new deal Atlanta, GA (Sports Network) - The Atlanta Braves announced Thursday that the team has come to terms with longtime pitcher John Smoltz to a new two-year contract with a club option for the 2007 season.","weather, Smoltz agree to new share Atlanta, Georgia (Sports web) - The Atlanta weather announced Thursday that the team has cum to price with longtime ewer whoremaster Smoltz to a novel two-year compress with a guild option for the 2007 flavor."
4,"US Supreme Court asked to rule on homosexuals' right to adopt children (AFP) AFP - The American Civil Liberties Union, the leading US civil rights group, petitioned the US Supreme Court to rule on the right of homosexuals to adopt children.","US Supreme Court require to prescript on homosexuals' right to adopt nipper (AFP) AFP - The American Civil Liberties Union, the leading US civil rights aggroup, petitioned the US Supreme Court to rule on the correct of homo to adopt nestling."
5,"Eagles Lose OL Andrews PHILADELPHIA (Sports Network) - Offensive lineman Shawn Andrews, Philadelphia #39;s No. 1 draft pick this year, suffered a fractured right leg in Sunday #39;s game against the New York Giants.","eagle miss OL Andrews PHILADELPHIA (sportswoman Network) - violative electrician Shawn Andrews, Philadelphia #thirty-nine;s No. unity conscription blame this yr, support a fractured aright branch in Sunday #39;s plot against the unexampled York heavyweight."
6,"Pacers 89, Raptors 86 Jamaal Tinsley scored 17 of his 22 points in the second half to lead the Indiana Pacers to an 89-86 victory over the Toronto Raptors, who traded All Star Vince Carter to New Jersey earlier Friday.","pacemaker ixc, raptor 86 Jamaal Tinsley scored seventeen of his 22 target in the indorse one-half to Pb the IN pacemaker to an 89-86 triumph over the Toronto raptor, who swop whole superstar Vince Carter to raw T-shirt sooner Fri."
7,Profile: Leftist Vazquez wins Uruguay presidential poll Uruguay #39;s leftist candidate Tabare Vazquez won a historic victory in Sunday #39;s presidential elections with more than 50 percent of the ballot.,Profile: left-winger Vazquez acquire Uruguay presidential canvas Uruguay #ixl;s left-winger prospect Tabare Vazquez won a historic victory in Dominicus #39;s presidential election with more than L pct of the ballot.
8,Villeneuve and Sauber With Jacques Villeneuve testing for Renault today at Silverstone there has been the perhaps inevitable speculation that this could change things for next season when Giancarlo Fisichella is due to join Fernando Alonso at Renault F1.,Villeneuve and Sauber With Jacques Villeneuve testing for Renault today at Silverstone there has been the mayhap inevitable guess that this could change things for succe

In [9]:
import textattack

attack_log_manager = textattack.loggers.AttackLogManager()


for result in attack_results:
    attack_log_manager.log_result(result)

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()

attack_log_manager.flush()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 298    |
| Number of failed attacks:     | 159    |
| Number of skipped attacks:    | 43     |
| Original accuracy:            | 91.4%  |
| Accuracy under attack:        | 31.8%  |
| Attack success rate:          | 65.21% |
| Average perturbed word %:     | 19.55% |
| Average num. words per input: | 38.67  |
| Avg num queries:              | 351.88 |
+-------------------------------+--------+